## Basic Usage

With some fake data.

In [1]:
from jsonlitedb import JSONLiteDB

In [2]:
db = JSONLiteDB(":memory:")
# more generally:
# db = JSONLiteDB('my_data.db')

Insert some data. Can use `insert()` with any number of items or `insertmany()` with an iterable (`insertmany([...]) <--> insert(*[...])`).

Can also use a context manager (`with db: ...`)to batch the insertions (or deletions).

In [3]:
db.insert(
    {"first": "John", "last": "Lennon", "born": 1940, "role": "guitar"},
    {"first": "Paul", "last": "McCartney", "born": 1942, "role": "bass"},
    {"first": "George", "last": "Harrison", "born": 1943, "role": "guitar"},
    {"first": "Ringo", "last": "Starr", "born": 1940, "role": "drums"},
    {"first": "George", "last": "Martin", "born": 1926, "role": "producer"},
)

In [4]:
len(db)

5

In [5]:
list(db)

[{'first': 'John', 'last': 'Lennon', 'born': 1940, 'role': 'guitar'},
 {'first': 'Paul', 'last': 'McCartney', 'born': 1942, 'role': 'bass'},
 {'first': 'George', 'last': 'Harrison', 'born': 1943, 'role': 'guitar'},
 {'first': 'Ringo', 'last': 'Starr', 'born': 1940, 'role': 'drums'},
 {'first': 'George', 'last': 'Martin', 'born': 1926, 'role': 'producer'}]

### Simple Queries

Let's do some simple queries. The default `query()` returns an iterator so we wrap them in a list.

In [6]:
list(db.query(first="George"))

[{'first': 'George', 'last': 'Harrison', 'born': 1943, 'role': 'guitar'},
 {'first': 'George', 'last': 'Martin', 'born': 1926, 'role': 'producer'}]

In [7]:
list(db.query(first="George", last="Martin"))

[{'first': 'George', 'last': 'Martin', 'born': 1926, 'role': 'producer'}]

Now let's query with a dictionary to match

In [8]:
list(db.query({"first": "George"}))

[{'first': 'George', 'last': 'Harrison', 'born': 1943, 'role': 'guitar'},
 {'first': 'George', 'last': 'Martin', 'born': 1926, 'role': 'producer'}]

Multiples are always an AND query

In [9]:
list(db.query({"first": "George", "last": "Martin"}))

[{'first': 'George', 'last': 'Martin', 'born': 1926, 'role': 'producer'}]

Can do seperate items but it makes no difference.

In [10]:
list(db.query({"first": "George"}, {"last": "Martin"}))

[{'first': 'George', 'last': 'Martin', 'born': 1926, 'role': 'producer'}]

### Query Objects

Query objects enable more complex combinations and inequalities. Query objects can be from the database (`db.Query` or `db.Q`) or created on thier own (`Query()` or `Q()`). They are all the same. 

In [11]:
list(db.query(db.Q.first == "George"))

[{'first': 'George', 'last': 'Harrison', 'born': 1943, 'role': 'guitar'},
 {'first': 'George', 'last': 'Martin', 'born': 1926, 'role': 'producer'}]

Note that you need to be careful with parentheses as the operator precedance for the `&` and `|` are very high

In [12]:
list(db.query((db.Q.first == "George") & (db.Q.last == "Martin")))

[{'first': 'George', 'last': 'Martin', 'born': 1926, 'role': 'producer'}]

Can do inequalities too

In [13]:
list(db.query(db.Q.born < 1930))

[{'first': 'George', 'last': 'Martin', 'born': 1926, 'role': 'producer'}]

Queries support: `==`, `!=`, `<`, `<=`, `>`, `>=` for normal comparisons.

In addition they support

- `%` : `LIKE`
- `*` : `GLOB`
- `@` : `REGEXP` using Python's regex module


In [14]:
# This will all be the same
db.query(db.Q.role % "prod%").all()  # LIKE
db.query(db.Q.role * "prod*").all()  # GLOB
db.query(db.Q.role @ "prod").all()  # REGEXP -- Python based

[{'first': 'George', 'last': 'Martin', 'born': 1926, 'role': 'producer'}]

### Speeding up queries

Queries can be **greatly accelerated** with an index. Note that SQLite is *extremely* picky about how you write the index! For the most part, if you the same method to query as write the index, you will be fine. (This is more of an issue with nested queries and *advanced* formulating of the query).

The name of the index is imaterial. It is based on the fields. It will look different

In [15]:
db.create_index("last")
db.indexes

{'ix_items_1bd45eb5': ['$."last"']}

In [16]:
# of course, with four items, this makes little difference
list(db.query(last="Martin"))

[{'first': 'George', 'last': 'Martin', 'born': 1926, 'role': 'producer'}]

And an index can also be used to enforce uniqueness amongst one or more fields

In [17]:
db.create_index("first", "last", unique=True)
db.indexes

{'ix_items_1bd45eb5': ['$."last"'],
 'ix_items_250e4243_UNIQUE': ['$."first"', '$."last"']}

In [18]:
# db.insert({'first': 'George', 'last': 'Martin', 'type':'FAKE ENTRY'})
# Causes: IntegrityError: UNIQUE constraint failed: index 'ix_items_250e4243_UNIQUE'

See *Advanced Usage* for more examples including nested queries